In [1]:
import pandas as pd
from functions import custom_cv_eval_v2
import random
import json


In [2]:
%run MODEL_LIST.ipynb            #Load the models

In [3]:
df=pd.read_csv("data/owl-with-odds.csv")

In [4]:
#The odds need to be changed to numeric...

subset = ['t1_odds', 't2_odds']
df.dropna(subset=subset ,inplace=True)
df['t1_odds'] = pd.to_numeric(df['t1_odds'], errors='coerce')
df['t2_odds'] = pd.to_numeric(df['t2_odds'], errors='coerce')
df.dropna(subset=subset ,inplace=True)
#Create the odds df
odds_df = df[subset]

#Create a label df
label_df = df['winner_label']

In [5]:
#Split the test set.  We are always(?) going to use the last 60 matches as the test set, so we don't want those around
#as we pick models
df_train = df[:-60]
odds_train = odds_df[:-60]
label_train = label_df[:-60]

df_test = df[-60:]
odds_test = odds_df[-60:]
label_test = label_df[-60:]

print(len(df_test))
print(len(odds_test))
print(len(label_test))

print(len(df_train))
print(len(odds_train))
print(len(label_train))

60
60
60
508
508
508


In [6]:
model_1

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10000,
                   multi_class='auto', n_jobs=None, penalty='none',
                   random_state=75, solver='saga', tol=0.0001, verbose=0,
                   warm_start=False)

In [7]:
def get_ev(input_df, input_model, input_features, input_labels, odds_input):
    df_sel = input_df[input_features]
    df_sel = df_sel.dropna()
    labels_sel = input_labels[input_labels.index.isin(df_sel.index)]
    odds_sel = odds_input[odds_input.index.isin(df_sel.index)] 
    best_score = custom_cv_eval_v2(df_sel, input_model, labels_sel, odds_sel)
    return best_score

In [8]:
def tune_LogisticRegression(input_model, input_features, input_df, input_labels, odds_input):
    ###############################################################################################################
    #Parameters we are going to fine-tune:
    #1. penalty ('l1' or 'l2')
    #2. tol (original_value, original_value * 1.2, original_value * 0.8, rand(0, 10)
    #3. random_state = 75
    #4. solver = 'newton-cg', 'lbfgs', 'sag', 'saga'    
    ###############################################################################################################
    output_model = input_model
    best_score = -1000
    penalty = ['l1', 'l2', 'none']
    solver = ['newton-cg', 'lbfgs', 'sag', 'saga']
    tol = [input_model.tol, input_model.tol * 1.2, input_model.tol * .8, random.random() * 10 ]
    for s in solver:
        score = -10000
        for p in penalty:
            for t in tol:
                if ((s == 'newton-cg') & (p == 'l1')) |\
                ((s == 'lbfgs') & (p == 'l1')) |\
                ((s == 'sag') & (p == 'l1')):

                    pass
                else:
                    test_model = LogisticRegression(solver = s, penalty = p, tol=t, random_state=75, max_iter=10000)
                    score = get_ev(input_df, test_model, input_features, input_labels, odds_input)
                    if score > best_score:
                        best_score = score
                        output_model = test_model
                        print(p, s, t, best_score)
    return(output_model)

In [40]:
def tune_DecisionTreeClassifier(input_model, input_features, input_df, input_labels, odds_input):
    ###############################################################################################################
    #Parameters we are going to fine-tune:
    #1. criterion ('gini', 'entropy')
    #2. splitter ('random', 'best')
    #3. max_depth ('none', IF A NUMBER EXISTS +2, +4, -2, -4 ELSE 4 RANDOM INTS 1->100)
    #4. min_samples_leaf(n-1, n-2, 0,  n+1, n+2)
    #5. max_leaf_nodes:('none', n+1, n+2, n-1, n-2, OR 4 random numbers)
    ###############################################################################################################
    output_model = input_model
    best_score = -1000
    criterion = ['gini', 'entropy']
    splitter = ['random', 'best']
    if input_model.max_depth == None:
        max_depth = [None, random.randrange(100), random.randrange(100), random.randrange(100), random.randrange(100)]
    else:
        max_depth = [input_model.max_depth, input_model.max_depth - 2, input_model.max_depth - 1,  
                     input_model.max_depth + 1, input_model.max_depth + 2, random.randrange(100)]
        max_depth = [i for i in max_depth if i > 0]

    min_samples_leaf = [input_model.min_samples_leaf, input_model.min_samples_leaf - 2, input_model.min_samples_leaf - 1,
                         input_model.min_samples_leaf + 1, input_model.min_samples_leaf + 2, random.randrange(100)]
    min_samples_leaf = [i for i in min_samples_leaf if i > 0]    
    if input_model.max_leaf_nodes == None:
        max_leaf_nodes = [None, random.randrange(1000), random.randrange(1000), random.randrange(1000), random.randrange(1000)]
    else:
        max_leaf_nodes = [input_model.max_leaf_nodes, input_model.max_leaf_nodes - 2, input_model.max_leaf_nodes - 1, 
                     input_model.max_leaf_nodes + 1, input_model.max_leaf_nodes + 2, random.randrange(1000)]
        max_leaf_nodes = [i for i in max_leaf_nodes if i > 0]
    
    for l in max_leaf_nodes:
        for sam in min_samples_leaf:
            for m in max_depth:
                for c in criterion:
                    for s in splitter:
                        test_model = DecisionTreeClassifier(criterion = c, splitter = s, max_depth = m,
                                                            min_samples_leaf=sam, max_leaf_nodes = l, random_state=75)
                        score = get_ev(input_df, test_model, input_features, input_labels, odds_input)
                        if score > best_score:
                            best_score = score
                            output_model = test_model
                            print("Criterion:", c, "splitter:", s, "max_depth:", m, 
                                  "min_samples_leaf:", sam, "max_leaf_nodes:", l, best_score)        
    
    
    
    return output_model




In [48]:
def tune_RandomForestClassifier(input_model, input_features, input_df, input_labels, odds_input):
    ###############################################################################################################
    #Parameters we are going to fine-tune:
    #1. criterion ('gini', 'entropy')
    #2. max_features ('auto', 'sqrt', 'log2')
    #3. max_depth ('none', IF A NUMBER EXISTS +2, +4, -2, -4 ELSE 4 RANDOM INTS 1->100)
    #4. min_samples_leaf(n-1, n-2, 0,  n+1, n+2)
    #5. max_leaf_nodes:('none', n+1, n+2, n-1, n-2, OR 4 random numbers)
    #6. n_estimators: (n, n+1, n+2, n-1, n-2)
    ###############################################################################################################    
    output_model = input_model
    best_score = get_ev(input_df, input_model, input_features, input_labels, odds_input)
    #1. criterion ('gini', 'entropy')
    criterion = ['gini', 'entropy']
    #2. max_features ('auto', 'sqrt', 'log2')
    max_features = ['auto', 'sqrt', 'log2', None]
    #3. max_depth ('none', IF A NUMBER EXISTS +2, +4, -2, -4 ELSE 4 RANDOM INTS 1->100)
    if input_model.max_depth == None:
        max_depth = [None, random.randrange(100), random.randrange(100), random.randrange(100), random.randrange(100)]
    else:
        max_depth = [input_model.max_depth, input_model.max_depth - 2, input_model.max_depth - 1,  
                     input_model.max_depth + 1, input_model.max_depth + 2, random.randrange(100)]
        max_depth = [i for i in max_depth if i > 0]
    #4. min_samples_leaf(n-1, n-2, 0,  n+1, n+2)
    min_samples_leaf = [input_model.min_samples_leaf, input_model.min_samples_leaf - 2, input_model.min_samples_leaf - 1,
                         input_model.min_samples_leaf + 1, input_model.min_samples_leaf + 2, random.randrange(100)]
    min_samples_leaf = [i for i in min_samples_leaf if i > 0]
    
    #5. max_leaf_nodes:('none', n+1, n+2, n-1, n-2, OR 4 random numbers)
    if input_model.max_leaf_nodes == None:
        max_leaf_nodes = [None, random.randrange(1000), random.randrange(1000), random.randrange(1000), random.randrange(1000)]
    else:
        max_leaf_nodes = [input_model.max_leaf_nodes, input_model.max_leaf_nodes - 2, input_model.max_leaf_nodes - 1, 
                     input_model.max_leaf_nodes + 1, input_model.max_leaf_nodes + 2, random.randrange(1000)]
        max_leaf_nodes = [i for i in max_leaf_nodes if i > 0]
    n_estimators = [input_model.n_estimators, input_model.n_estimators - 2, input_model.n_estimators - 1,  
                 input_model.n_estimators + 1, input_model.n_estimators + 2, random.randrange(200)]
    n_estimators = [i for i in n_estimators if i > 0]
    
    
    
    for n in n_estimators:
        for ml in max_leaf_nodes:
            for ms in min_samples_leaf:
                for md in max_depth:
                    for mf in max_features:
                        for c in criterion:
                            test_model = RandomForestClassifier(n_estimators = n, max_leaf_nodes = ml, 
                                                                min_samples_leaf = ms,
                                                                max_depth = md, criterion = c, 
                                                                max_features = mf, random_state=75)
                            score = get_ev(input_df, test_model, input_features, input_labels, odds_input)
                            if score > best_score:
                                best_score = score
                                output_model = test_model
                                print()
                                print("NEW BEST SCORE")
                                print("Criterion:", c, "max_features:", mf, "max_depth:", md, "min_samples_leaf:", ms,
                                      "max_leaf_nodes:", ml, "n_estimators", n, best_score)        
                                print()
                                print()
                            else:
                                print("Criterion:", c, "max_features:", mf, "max_depth:", md, "min_samples_leaf:", ms,
                                      "max_leaf_nodes:", ml, "n_estimators", n, score)        
                            
    return output_model

In [49]:
def tune_hyperparameters(input_model, input_features, input_df, input_labels, odds_input):
    best_model = input_model
    keep_going = True
    if isinstance(input_model, LogisticRegression):
        while(keep_going):
            pos_model = (tune_LogisticRegression(input_model, input_features, input_df, input_labels, odds_input))
            
            if str(pos_model) == str(best_model):  #Direct comparisons don't seem to work....
                keep_going = False
                output_model = best_model
            else:
                best_model = pos_model

    elif isinstance(input_model, DecisionTreeClassifier):
        while(keep_going):
            pos_model = (tune_DecisionTreeClassifier(best_model, input_features, input_df, input_labels, odds_input))
            if str(pos_model) == str(best_model):  #Direct comparisons don't seem to work....
                keep_going = False
                output_model = best_model
            else:
                best_model = pos_model            
                
    elif isinstance(input_model, RandomForestClassifier):
        while(keep_going):
            pos_model = (tune_RandomForestClassifier(best_model, input_features, input_df, input_labels, odds_input))
            if str(pos_model) == str(best_model):  #Direct comparisons don't seem to work....
                keep_going = False
                output_model = best_model
            else:
                best_model = pos_model            
        
    else:
        output_model = input_model
    return(output_model)

In [50]:
tune_hyperparameters(model_3, features_3, df_train, label_train, odds_train)

Criterion: gini max_features: auto max_depth: None min_samples_leaf: 1 max_leaf_nodes: None n_estimators 100 141.9555746512028
Criterion: entropy max_features: auto max_depth: None min_samples_leaf: 1 max_leaf_nodes: None n_estimators 100 129.34757114859815
Criterion: gini max_features: sqrt max_depth: None min_samples_leaf: 1 max_leaf_nodes: None n_estimators 100 141.9555746512028
Criterion: entropy max_features: sqrt max_depth: None min_samples_leaf: 1 max_leaf_nodes: None n_estimators 100 129.34757114859815
Criterion: gini max_features: log2 max_depth: None min_samples_leaf: 1 max_leaf_nodes: None n_estimators 100 141.9555746512028
Criterion: entropy max_features: log2 max_depth: None min_samples_leaf: 1 max_leaf_nodes: None n_estimators 100 129.34757114859815

NEW BEST SCORE
Criterion: gini max_features: None max_depth: None min_samples_leaf: 1 max_leaf_nodes: None n_estimators 100 146.1505454262852


Criterion: entropy max_features: None max_depth: None min_samples_leaf: 1 max_lea

Criterion: entropy max_features: auto max_depth: 59 min_samples_leaf: 2 max_leaf_nodes: None n_estimators 100 101.32764879335303
Criterion: gini max_features: sqrt max_depth: 59 min_samples_leaf: 2 max_leaf_nodes: None n_estimators 100 101.30266276979636
Criterion: entropy max_features: sqrt max_depth: 59 min_samples_leaf: 2 max_leaf_nodes: None n_estimators 100 101.32764879335303
Criterion: gini max_features: log2 max_depth: 59 min_samples_leaf: 2 max_leaf_nodes: None n_estimators 100 101.30266276979636
Criterion: entropy max_features: log2 max_depth: 59 min_samples_leaf: 2 max_leaf_nodes: None n_estimators 100 101.32764879335303
Criterion: gini max_features: None max_depth: 59 min_samples_leaf: 2 max_leaf_nodes: None n_estimators 100 98.62704736213695
Criterion: entropy max_features: None max_depth: 59 min_samples_leaf: 2 max_leaf_nodes: None n_estimators 100 106.16754458209472
Criterion: gini max_features: auto max_depth: 55 min_samples_leaf: 2 max_leaf_nodes: None n_estimators 100 

Criterion: gini max_features: sqrt max_depth: 52 min_samples_leaf: 28 max_leaf_nodes: None n_estimators 100 17.719382423402834
Criterion: entropy max_features: sqrt max_depth: 52 min_samples_leaf: 28 max_leaf_nodes: None n_estimators 100 17.76632935843588
Criterion: gini max_features: log2 max_depth: 52 min_samples_leaf: 28 max_leaf_nodes: None n_estimators 100 17.719382423402834
Criterion: entropy max_features: log2 max_depth: 52 min_samples_leaf: 28 max_leaf_nodes: None n_estimators 100 17.76632935843588
Criterion: gini max_features: None max_depth: 52 min_samples_leaf: 28 max_leaf_nodes: None n_estimators 100 29.36081373179773
Criterion: entropy max_features: None max_depth: 52 min_samples_leaf: 28 max_leaf_nodes: None n_estimators 100 33.659613228985215
Criterion: gini max_features: auto max_depth: 57 min_samples_leaf: 28 max_leaf_nodes: None n_estimators 100 17.719382423402834
Criterion: entropy max_features: auto max_depth: 57 min_samples_leaf: 28 max_leaf_nodes: None n_estimator

Criterion: entropy max_features: sqrt max_depth: 55 min_samples_leaf: 1 max_leaf_nodes: 657 n_estimators 100 127.86158292731169
Criterion: gini max_features: log2 max_depth: 55 min_samples_leaf: 1 max_leaf_nodes: 657 n_estimators 100 131.31196519884946
Criterion: entropy max_features: log2 max_depth: 55 min_samples_leaf: 1 max_leaf_nodes: 657 n_estimators 100 127.86158292731169
Criterion: gini max_features: None max_depth: 55 min_samples_leaf: 1 max_leaf_nodes: 657 n_estimators 100 145.7726124101041
Criterion: entropy max_features: None max_depth: 55 min_samples_leaf: 1 max_leaf_nodes: 657 n_estimators 100 146.13771058310286
Criterion: gini max_features: auto max_depth: None min_samples_leaf: 2 max_leaf_nodes: 657 n_estimators 100 93.57620119607581
Criterion: entropy max_features: auto max_depth: None min_samples_leaf: 2 max_leaf_nodes: 657 n_estimators 100 96.76430915774046
Criterion: gini max_features: sqrt max_depth: None min_samples_leaf: 2 max_leaf_nodes: 657 n_estimators 100 93.5

Criterion: gini max_features: log2 max_depth: 57 min_samples_leaf: 3 max_leaf_nodes: 657 n_estimators 100 87.38748097158401
Criterion: entropy max_features: log2 max_depth: 57 min_samples_leaf: 3 max_leaf_nodes: 657 n_estimators 100 91.37509896780813
Criterion: gini max_features: None max_depth: 57 min_samples_leaf: 3 max_leaf_nodes: 657 n_estimators 100 88.17697887140808
Criterion: entropy max_features: None max_depth: 57 min_samples_leaf: 3 max_leaf_nodes: 657 n_estimators 100 83.59301819401136
Criterion: gini max_features: auto max_depth: 59 min_samples_leaf: 3 max_leaf_nodes: 657 n_estimators 100 87.38748097158401
Criterion: entropy max_features: auto max_depth: 59 min_samples_leaf: 3 max_leaf_nodes: 657 n_estimators 100 91.37509896780813
Criterion: gini max_features: sqrt max_depth: 59 min_samples_leaf: 3 max_leaf_nodes: 657 n_estimators 100 87.38748097158401
Criterion: entropy max_features: sqrt max_depth: 59 min_samples_leaf: 3 max_leaf_nodes: 657 n_estimators 100 91.37509896780

...

Criterion: gini max_features: None max_depth: None min_samples_leaf: 3 max_leaf_nodes: 237 n_estimators 4 65.32145836400723
Criterion: entropy max_features: None max_depth: None min_samples_leaf: 3 max_leaf_nodes: 237 n_estimators 4 60.02212119308143
Criterion: gini max_features: auto max_depth: 95 min_samples_leaf: 3 max_leaf_nodes: 237 n_estimators 4 25.21042645722233
Criterion: entropy max_features: auto max_depth: 95 min_samples_leaf: 3 max_leaf_nodes: 237 n_estimators 4 22.002716719219364
Criterion: gini max_features: sqrt max_depth: 95 min_samples_leaf: 3 max_leaf_nodes: 237 n_estimators 4 25.21042645722233
Criterion: entropy max_features: sqrt max_depth: 95 min_samples_leaf: 3 max_leaf_nodes: 237 n_estimators 4 22.002716719219364
Criterion: gini max_features: log2 max_depth: 95 min_samples_leaf: 3 max_leaf_nodes: 237 n_estimators 4 25.21042645722233
Criterion: entropy max_features: log2 max_depth: 95 min_samples_leaf: 3 max_leaf_nodes: 237 n_estimators 4 22.002716719219364
Crite

Criterion: gini max_features: auto max_depth: 90 min_samples_leaf: 13 max_leaf_nodes: 237 n_estimators 4 30.617223803347684
Criterion: entropy max_features: auto max_depth: 90 min_samples_leaf: 13 max_leaf_nodes: 237 n_estimators 4 36.318330602653774
Criterion: gini max_features: sqrt max_depth: 90 min_samples_leaf: 13 max_leaf_nodes: 237 n_estimators 4 30.617223803347684
Criterion: entropy max_features: sqrt max_depth: 90 min_samples_leaf: 13 max_leaf_nodes: 237 n_estimators 4 36.318330602653774
Criterion: gini max_features: log2 max_depth: 90 min_samples_leaf: 13 max_leaf_nodes: 237 n_estimators 4 30.617223803347684
Criterion: entropy max_features: log2 max_depth: 90 min_samples_leaf: 13 max_leaf_nodes: 237 n_estimators 4 36.318330602653774
Criterion: gini max_features: None max_depth: 90 min_samples_leaf: 13 max_leaf_nodes: 237 n_estimators 4 37.605578217117774
Criterion: entropy max_features: None max_depth: 90 min_samples_leaf: 13 max_leaf_nodes: 237 n_estimators 4 51.223908224073

Criterion: entropy max_features: sqrt max_depth: 61 min_samples_leaf: 2 max_leaf_nodes: 834 n_estimators 4 23.716706830170114
Criterion: gini max_features: log2 max_depth: 61 min_samples_leaf: 2 max_leaf_nodes: 834 n_estimators 4 13.381214678238209
Criterion: entropy max_features: log2 max_depth: 61 min_samples_leaf: 2 max_leaf_nodes: 834 n_estimators 4 23.716706830170114
Criterion: gini max_features: None max_depth: 61 min_samples_leaf: 2 max_leaf_nodes: 834 n_estimators 4 43.31971904598795
Criterion: entropy max_features: None max_depth: 61 min_samples_leaf: 2 max_leaf_nodes: 834 n_estimators 4 54.62197340249132
Criterion: gini max_features: auto max_depth: 43 min_samples_leaf: 2 max_leaf_nodes: 834 n_estimators 4 13.381214678238209
Criterion: entropy max_features: auto max_depth: 43 min_samples_leaf: 2 max_leaf_nodes: 834 n_estimators 4 23.716706830170114
Criterion: gini max_features: sqrt max_depth: 43 min_samples_leaf: 2 max_leaf_nodes: 834 n_estimators 4 13.381214678238209
Criter

Criterion: entropy max_features: None max_depth: None min_samples_leaf: 13 max_leaf_nodes: 834 n_estimators 4 51.223908224073
Criterion: gini max_features: auto max_depth: 95 min_samples_leaf: 13 max_leaf_nodes: 834 n_estimators 4 30.617223803347684
Criterion: entropy max_features: auto max_depth: 95 min_samples_leaf: 13 max_leaf_nodes: 834 n_estimators 4 36.318330602653774
Criterion: gini max_features: sqrt max_depth: 95 min_samples_leaf: 13 max_leaf_nodes: 834 n_estimators 4 30.617223803347684
Criterion: entropy max_features: sqrt max_depth: 95 min_samples_leaf: 13 max_leaf_nodes: 834 n_estimators 4 36.318330602653774
Criterion: gini max_features: log2 max_depth: 95 min_samples_leaf: 13 max_leaf_nodes: 834 n_estimators 4 30.617223803347684
Criterion: entropy max_features: log2 max_depth: 95 min_samples_leaf: 13 max_leaf_nodes: 834 n_estimators 4 36.318330602653774
Criterion: gini max_features: None max_depth: 95 min_samples_leaf: 13 max_leaf_nodes: 834 n_estimators 4 37.6055782171177

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features=None,
                       max_leaf_nodes=235, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=97,
                       n_jobs=None, oob_score=False, random_state=75, verbose=0,
                       warm_start=False)

In [11]:
tune_hyperparameters(model_2, features_2, df_train, label_train, odds_train)

Criterion: gini splitter: random max_depth: 13 min_samples_leaf: 1 max_leaf_nodes: 144 73.10984269006656
Criterion: entropy splitter: random max_depth: 13 min_samples_leaf: 1 max_leaf_nodes: 144 120.09023623685835


DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=13, max_features=None, max_leaf_nodes=144,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=75, splitter='random')

In [13]:
tune_hyperparameters(model_1, features_1, df_train, label_train, odds_train)

l2 newton-cg 0.0001 53.06760226712672
none newton-cg 0.0001 55.4021265248508
none saga 0.0001 55.607690050410156


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10000,
                   multi_class='auto', n_jobs=None, penalty='none',
                   random_state=75, solver='saga', tol=0.0001, verbose=0,
                   warm_start=False)